In [1]:
import numpy as np
import tensorflow as tf
from neuralnetworks import ffnn, embedding, attention, masking, transformer
from tokenization import tokenizer
import importlib

2024-12-08 13:34:42.247359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
size = 2**8

size

256

In [5]:
embedding = embedding.EmbeddingLayer(size, name = "embedding")

In [6]:
input_string = "the cow jumped over the moon"

tkn = tokenizer.Tokenizer()

tkn(input_string)

In [9]:
for t in tkn.token_q:
    embedding(t)

In [23]:
#this is where positional encoding should happen

batch = tf.expand_dims(embedding.embeddings, axis = 0)

In [29]:
importlib.reload(attention)

tfmrs = {}

for i in range (1,33):
    tfmrs = {**tfmrs, **{i: transformer.TransformerLayer(size, batch.shape[1], 4 , 4*size)}}

tfmrs

{1: <neuralnetworks.transformer.TransformerLayer at 0x16136f940>,
 2: <neuralnetworks.transformer.TransformerLayer at 0x161c6eeb0>,
 3: <neuralnetworks.transformer.TransformerLayer at 0x161c7dcd0>,
 4: <neuralnetworks.transformer.TransformerLayer at 0x161c8eaf0>,
 5: <neuralnetworks.transformer.TransformerLayer at 0x161c9e910>,
 6: <neuralnetworks.transformer.TransformerLayer at 0x161cad730>,
 7: <neuralnetworks.transformer.TransformerLayer at 0x161cbc550>,
 8: <neuralnetworks.transformer.TransformerLayer at 0x161cce3a0>,
 9: <neuralnetworks.transformer.TransformerLayer at 0x161cdf1c0>,
 10: <neuralnetworks.transformer.TransformerLayer at 0x161ce9fa0>,
 11: <neuralnetworks.transformer.TransformerLayer at 0x161cf8dc0>,
 12: <neuralnetworks.transformer.TransformerLayer at 0x161d08be0>,
 13: <neuralnetworks.transformer.TransformerLayer at 0x161d19a00>,
 14: <neuralnetworks.transformer.TransformerLayer at 0x161d29820>,
 15: <neuralnetworks.transformer.TransformerLayer at 0x161d3b640>,
 16:

In [34]:
for i in range(1,33):
    batch = tfmrs[i](batch)

batch

<tf.Tensor: shape=(1, 5, 256), dtype=float32, numpy=
array([[[-0.09947905,  0.72585285,  0.05796956, ...,  0.47017947,
         -0.29539815, -1.2876405 ],
        [-0.63762605,  0.25631022, -0.8624704 , ...,  1.2280862 ,
         -1.3891668 , -0.55693626],
        [-0.1854893 ,  0.40731052, -0.89137244, ...,  1.1436095 ,
         -0.71106625, -0.28736824],
        [-0.5650112 ,  0.6678289 , -0.3698084 , ...,  1.2423675 ,
          0.76084447, -0.5305328 ],
        [ 0.07580578,  0.43767974,  0.18874934, ...,  1.1018841 ,
         -0.23056355, -0.42522687]]], dtype=float32)>

In [37]:
class TransformerFinalLayer(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super(TransformerFinalLayer, self).__init__()
        self.projection = tf.keras.layers.Dense(vocab_size)

    def __call__(self, inputs):
        logits = self.projection(inputs)  # Project to vocab_size
        probabilities = tf.nn.softmax(logits, axis=-1)  # Convert to probabilities
        return probabilities


In [45]:
final_layer = TransformerFinalLayer(10, batch.shape[2])

In [46]:
final_layer(batch)

<tf.Tensor: shape=(1, 5, 10), dtype=float32, numpy=
array([[[0.03044865, 0.01383561, 0.20512821, 0.01574093, 0.1889186 ,
         0.03273387, 0.04645465, 0.22605082, 0.1199559 , 0.12073276],
        [0.02632543, 0.02578001, 0.2321132 , 0.09279155, 0.27085042,
         0.0603786 , 0.14260237, 0.05851465, 0.02639292, 0.06425083],
        [0.15056264, 0.03874962, 0.17088121, 0.06529535, 0.13705961,
         0.07896155, 0.09057087, 0.10327346, 0.06807107, 0.09657457],
        [0.04188018, 0.06694464, 0.08726914, 0.04041603, 0.15776102,
         0.03249858, 0.07331317, 0.10460029, 0.15982619, 0.2354908 ],
        [0.0265914 , 0.1450832 , 0.04418058, 0.21870838, 0.111321  ,
         0.13693565, 0.03594118, 0.04137687, 0.15984032, 0.08002143]]],
      dtype=float32)>

# Vocabulary Initialization

In [43]:
with open("/Users/joericks/Desktop/nova/model/constants/vocabulary.txt", "r") as f:
    vocab = f.read().split('\n')
    print(vocab)

['#define', '#include', '#if', '#else', '#endif', 'auto', 'break', 'case', 'char', 'const', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extern', 'float', 'for', 'goto', 'if', 'int', 'long', 'register', 'return', 'short', 'signed', 'sizeof', 'static', 'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while', '_Bool', '_Complex', '_Imaginary', 'inline', 'restrict', '_Alignas', '_Alignof', '_Atomic', '_Generic', '_Noreturn', '_Static_assert', '_Thread_local', '+', '-', '*', '/', '%', '==', '!=', '>', '<', '>=', '<=', '&&', '||', '!', '&', '|', '^', '~', '<<', '>>', '++', '--', '*', '&', 'sizeof', '?:', ',', '.', '->', '%int', '%var', '%float', '%string', '%func', '%class', ';', '{', '}', '{}', '']


In [44]:
vocab_size = len(vocab)

In [78]:
new_final_layer = TransformerFinalLayer(vocab_size, batch.shape[2])

In [79]:
def mean(arr):
    return sum([i*arr[i] for i in range(0, len(arr))])

In [82]:
final = new_final_layer(batch)
final

<tf.Tensor: shape=(1, 5, 89), dtype=float32, numpy=
array([[[0.00978142, 0.0065435 , 0.00821909, 0.00429984, 0.02690102,
         0.0061669 , 0.01417588, 0.0050653 , 0.00396573, 0.02046988,
         0.00776417, 0.01684113, 0.01805889, 0.005921  , 0.0077371 ,
         0.01003014, 0.0110882 , 0.01500982, 0.00334477, 0.00905581,
         0.02510174, 0.00771801, 0.00572649, 0.00161598, 0.02371358,
         0.02326424, 0.00909672, 0.01462158, 0.02120908, 0.013144  ,
         0.00205382, 0.00978873, 0.00560785, 0.00257272, 0.01867482,
         0.00901906, 0.00361429, 0.00991141, 0.00912776, 0.02127976,
         0.01272175, 0.00336248, 0.01069815, 0.00403742, 0.00513064,
         0.01039648, 0.00496457, 0.00771462, 0.04948825, 0.0235945 ,
         0.0255486 , 0.00627909, 0.00449078, 0.00976756, 0.01833355,
         0.02072923, 0.0026971 , 0.01644593, 0.0054278 , 0.01181918,
         0.01125352, 0.00982694, 0.00453483, 0.01572102, 0.00467145,
         0.03364301, 0.01184075, 0.002803  , 0.0060

In [81]:
means = []

for i in range(final.shape[0]):
    for j in range(final.shape[1]):
        print(mean(final[i,j]))
        means.append(mean(final[i,j]))

tf.Tensor(43.136044, shape=(), dtype=float32)
tf.Tensor(40.895535, shape=(), dtype=float32)
tf.Tensor(43.852276, shape=(), dtype=float32)
tf.Tensor(43.80238, shape=(), dtype=float32)
tf.Tensor(43.360542, shape=(), dtype=float32)


In [77]:
means

[<tf.Tensor: shape=(), dtype=float32, numpy=45.7917>,
 <tf.Tensor: shape=(), dtype=float32, numpy=43.750305>,
 <tf.Tensor: shape=(), dtype=float32, numpy=46.327908>,
 <tf.Tensor: shape=(), dtype=float32, numpy=46.486794>,
 <tf.Tensor: shape=(), dtype=float32, numpy=45.664543>]